# AutoEncoder Model Architecture Demo

This notebook demonstrates how to initialize and use different autoencoder architectures from the `autoencoder_lib` package for 64x64 grayscale images.

## Available Architectures

1. **SimpleLinearAutoencoder**: Basic fully-connected architecture
2. **DeeperLinearAutoencoder**: Deeper fully-connected architecture  
3. **ConvAutoencoder**: Basic convolutional architecture
4. **DeeperConvAutoencoder**: Advanced convolutional architecture

In [1]:
import torch
import numpy as np
from autoencoder_lib.models import (
    SimpleLinearAutoencoder, 
    DeeperLinearAutoencoder,
    ConvAutoencoder, 
    DeeperConvAutoencoder,
    ModelFactory,
    create_autoencoder
)

print("Successfully imported all model architectures!")
print(f"Available architectures: {ModelFactory.get_available_architectures()}")

Successfully imported all model architectures!
Available architectures: ['simple_linear', 'deeper_linear', 'convolutional', 'deeper_convolutional']


## Model Initialization Parameters

For 64x64 grayscale images:
- **Input size for linear models**: 64 × 64 = 4096 (flattened)
- **Input channels for conv models**: 1 (grayscale)
- **Latent dimension**: We'll use 16 for demonstration (adjustable)

In [2]:
# Parameters for 64x64 grayscale images
INPUT_SIZE = 64 * 64  # Flattened size for linear models
INPUT_CHANNELS = 1    # Grayscale images
LATENT_DIM = 16       # Latent space dimension
IMAGE_SHAPE = (1, 64, 64)  # (channels, height, width)

print(f"Input size (flattened): {INPUT_SIZE}")
print(f"Input channels: {INPUT_CHANNELS}")
print(f"Latent dimension: {LATENT_DIM}")
print(f"Image shape: {IMAGE_SHAPE}")

Input size (flattened): 4096
Input channels: 1
Latent dimension: 16
Image shape: (1, 64, 64)


## 1. Linear Autoencoder Architectures

Linear autoencoders flatten the input image and use fully-connected layers.

In [3]:
# Method 1: Direct instantiation
simple_linear = SimpleLinearAutoencoder(
    input_size=INPUT_SIZE, 
    latent_dim=LATENT_DIM
)

deeper_linear = DeeperLinearAutoencoder(
    input_size=INPUT_SIZE,
    latent_dim=LATENT_DIM
)

# Method 2: Using factory
simple_linear_factory = create_autoencoder(
    'simple_linear',
    input_size=INPUT_SIZE,
    latent_dim=LATENT_DIM
)

deeper_linear_factory = ModelFactory.create_model(
    'deeper_linear',
    input_size=INPUT_SIZE,
    latent_dim=LATENT_DIM
)

print("✅ Linear autoencoders initialized successfully!")
print(f"Simple Linear info: {simple_linear.get_model_info()}")
print(f"Deeper Linear info: {deeper_linear.get_model_info()}")

✅ Linear autoencoders initialized successfully!
Simple Linear info: {'model_class': 'SimpleLinearAutoencoder', 'latent_dim': 16, 'total_params': 1057040, 'trainable_params': 1057040, 'input_size': 4096, 'architecture_type': 'linear'}
Deeper Linear info: {'model_class': 'DeeperLinearAutoencoder', 'latent_dim': 16, 'total_params': 2186128, 'trainable_params': 2186128, 'input_size': 4096, 'architecture_type': 'linear'}


## 2. Convolutional Autoencoder Architectures

Convolutional autoencoders use CNN layers to preserve spatial relationships.

In [4]:
# Method 1: Direct instantiation
conv_autoencoder = ConvAutoencoder(
    input_channels=INPUT_CHANNELS,
    latent_dim=LATENT_DIM,
    input_size=(64, 64)
)

deeper_conv_autoencoder = DeeperConvAutoencoder(
    input_channels=INPUT_CHANNELS,
    latent_dim=LATENT_DIM,  # Note: DeeperConv typically uses larger latent_dim (default 32)
    input_size=(64, 64)
)

# Method 2: Using factory
conv_factory = create_autoencoder(
    'convolutional',
    input_channels=INPUT_CHANNELS,
    latent_dim=LATENT_DIM
)

deeper_conv_factory = ModelFactory.create_model(
    'deeper_convolutional',
    input_channels=INPUT_CHANNELS,
    latent_dim=LATENT_DIM
)

print("✅ Convolutional autoencoders initialized successfully!")
print(f"Conv info: {conv_autoencoder.get_model_info()}")
print(f"Deeper Conv info: {deeper_conv_autoencoder.get_model_info()}")

✅ Convolutional autoencoders initialized successfully!
Conv info: {'model_class': 'ConvAutoencoder', 'latent_dim': 16, 'total_params': 181713, 'trainable_params': 181713, 'input_channels': 1, 'input_size': (64, 64), 'architecture_type': 'convolutional'}
Deeper Conv info: {'model_class': 'DeeperConvAutoencoder', 'latent_dim': 16, 'total_params': 4992273, 'trainable_params': 4992273, 'input_channels': 1, 'input_size': (64, 64), 'architecture_type': 'convolutional'}


## 3. Model Comparison

Let's compare the parameter counts and architectures of all models:

In [5]:
models = {
    'SimpleLinearAutoencoder': simple_linear,
    'DeeperLinearAutoencoder': deeper_linear,
    'ConvAutoencoder': conv_autoencoder,
    'DeeperConvAutoencoder': deeper_conv_autoencoder
}

print("📊 Model Architecture Comparison")
print("=" * 50)

for name, model in models.items():
    info = model.get_model_info()
    params = info['total_params']
    arch_type = info.get('architecture_type', 'N/A')
    
    print(f"{name:25} | {params:>10,} params | {arch_type:>15} architecture")

print("=" * 50)

📊 Model Architecture Comparison
SimpleLinearAutoencoder   |  1,057,040 params |          linear architecture
DeeperLinearAutoencoder   |  2,186,128 params |          linear architecture
ConvAutoencoder           |    181,713 params |   convolutional architecture
DeeperConvAutoencoder     |  4,992,273 params |   convolutional architecture


## 4. Testing Forward Pass

Let's test each model with dummy 64x64 grayscale images to ensure they work correctly:

In [6]:
# Create dummy input data (batch of 4 images)
batch_size = 4
dummy_input = torch.randn(batch_size, 1, 64, 64)  # (batch, channels, height, width)

print(f"Input shape: {dummy_input.shape}")
print("\n🧪 Testing forward pass for each model:")
print("=" * 50)

for name, model in models.items():
    try:
        # Set model to evaluation mode
        model.eval()
        
        with torch.no_grad():
            # Forward pass: get encoded and decoded outputs
            encoded, decoded = model(dummy_input)
            
        print(f"✅ {name:25}")
        print(f"   Input:   {dummy_input.shape}")
        print(f"   Encoded: {encoded.shape}")  
        print(f"   Decoded: {decoded.shape}")
        print(f"   Latent dim: {encoded.shape[1]}")
        print()
        
    except Exception as e:
        print(f"❌ {name:25} - Error: {e}")
        print()

Input shape: torch.Size([4, 1, 64, 64])

🧪 Testing forward pass for each model:
✅ SimpleLinearAutoencoder  
   Input:   torch.Size([4, 1, 64, 64])
   Encoded: torch.Size([4, 16])
   Decoded: torch.Size([4, 1, 64, 64])
   Latent dim: 16

✅ DeeperLinearAutoencoder  
   Input:   torch.Size([4, 1, 64, 64])
   Encoded: torch.Size([4, 16])
   Decoded: torch.Size([4, 1, 64, 64])
   Latent dim: 16

✅ ConvAutoencoder          
   Input:   torch.Size([4, 1, 64, 64])
   Encoded: torch.Size([4, 16])
   Decoded: torch.Size([4, 1, 64, 64])
   Latent dim: 16

✅ DeeperConvAutoencoder    
   Input:   torch.Size([4, 1, 64, 64])
   Encoded: torch.Size([4, 16])
   Decoded: torch.Size([4, 1, 64, 64])
   Latent dim: 16



## 5. Using ModelFactory for Configuration-based Creation

The ModelFactory supports creating models from configuration dictionaries:

In [ ]:
# Example configurations for different models
configs = [
    {
        'architecture': 'simple_linear',
        'input_size': 64*64,
        'latent_dim': 32
    },
    {
        'architecture': 'convolutional', 
        'input_channels': 1,
        'latent_dim': 64
    },
    {
        'architecture': 'deeper_convolutional',
        'input_channels': 1,
        'latent_dim': 128,
        'input_size': (64, 64)
    }
]

print("🏭 Creating models from configuration dictionaries:")
print("=" * 55)

for i, config in enumerate(configs, 1):
    try:
        model = ModelFactory.create_model_from_config(config)
        info = model.get_model_info()
        
        print(f"Config {i}: {config['architecture']}")
        print(f"   ✅ Created successfully")
        print(f"   📊 Parameters: {info['total_params']:,}")
        print(f"   🔧 Latent dim: {info['latent_dim']}")
        print()
        
    except Exception as e:
        print(f"Config {i}: ❌ Error: {e}")
        print()

## 6. Architecture Information Summary

Get detailed information about each available architecture:

In [ ]:
print("📋 Detailed Architecture Information:")
print("=" * 60)

for arch_name in ModelFactory.get_available_architectures():
    try:
        info = ModelFactory.get_model_info(arch_name)
        
        print(f"\n🏗️  {arch_name.upper()}")
        print(f"   Class: {info['class_name']}")
        print(f"   Type: {info['architecture_type']}")
        print(f"   Parameters: {info['total_params']:,}")
        print(f"   Latent Dimension: {info['latent_dim']}")
        
        # Print description if available
        if info.get('docstring'):
            desc = info['docstring'].split('\n')[0].strip()
            if desc:
                print(f"   Description: {desc}")
        
    except Exception as e:
        print(f"❌ Error getting info for {arch_name}: {e}")

print("\n" + "=" * 60)
print("🎉 All model architectures are ready for experimentation!")